<a href="https://colab.research.google.com/github/kkkkh1210/gdg-ml-assignment/blob/main/week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Titanic: 최종 모델 선정 및 예측 제출 파이프라인

import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# 1. 데이터 불러오기
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/titanic/train.csv")
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/titanic/test.csv")


# 제출용 ID 저장
test_ids = test['PassengerId']

# 2. 전처리 함수 정의
def preprocess(df):
    df = df.copy()
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    df = df.drop(columns=['Cabin', 'Name', 'Ticket'])
    df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
    return df

train_proc = preprocess(train)
test_proc = preprocess(test)

X = train_proc.drop(columns=['Survived', 'PassengerId'])
y = train_proc['Survived']
X_test_final = test_proc.drop(columns=['PassengerId'])

# 3. 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test_final)

# 4. 모델 정의 및 성능 비교 (LightGBM 제외)
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

results = {}
for name, model in models.items():
    model.fit(X_scaled, y)
    scores = cross_val_score(model, X_scaled, y, cv=5, scoring='accuracy')
    results[name] = {
        'model': model,
        'mean_cv_accuracy': scores.mean()
    }

# 5. 최고 성능 모델 선택
best_model_name = max(results, key=lambda x: results[x]['mean_cv_accuracy'])
best_model = results[best_model_name]['model']
best_accuracy = results[best_model_name]['mean_cv_accuracy']

# 6. 테스트셋 예측 및 제출 파일 생성
preds = best_model.predict(X_test_scaled)
submission = pd.DataFrame({
    'PassengerId': test_ids,
    'Survived': preds
})

# 저장
filename = f"titanic_best_{best_model_name}.csv"
submission.to_csv(filename, index=False)
print(f"Best model: {best_model_name}, Accuracy: {best_accuracy:.4f}, File saved: {filename}")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:32:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:32:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:32:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:32:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best model: XGBoost, Accuracy: 0.8238, File saved: titanic_best_XGBoost.csv


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:32:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:32:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
